In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation
from bokeh.models import Button, CustomJS


In [2]:
recipricol_summary = pd.read_csv('country-by-time.csv')

recipricol_summary.date = pd.to_datetime(recipricol_summary.date, format="%Y-%m-%d")

recipricol_summary.set_index("date", inplace = True)

In [3]:
recipricol_summary.index.unique()

DatetimeIndex(['2025-02-04', '2025-03-06', '2025-03-12', '2025-04-03',
               '2025-04-05', '2025-04-08', '2025-04-09', '2025-04-11',
               '2025-05-12', '2025-06-03', '2025-08-07', '2025-11-01',
               '2025-11-14'],
              dtype='datetime64[ns]', name='date', freq=None)

In [4]:
foo = recipricol_summary.loc["2025-11-14"].copy(deep=True)

foo.sort_values(by='total imports', ascending=False, inplace=True)

foo[0:20].head(20)

foobar = foo[0:50].copy(deep=True)

foobar.rename({"effective tariff": "2025 tariff increase"}, axis=1, inplace=True)

avg_tariff = ( foobar["2025 tariff increase"]*foobar["total imports"] ).sum() / foobar["total imports"].sum() + 2.3 

print("Average Tariff Rate: ", avg_tariff)

Average Tariff Rate:  13.975266365491787


In [5]:
foobar.reset_index(inplace=True, drop=True)

In [6]:
foobar.head(20)

,country_name,2025 tariff increase,total imports,2024 tariff
0,EUROPEAN UNION,11.129221,5.974732e+11,1.229078
1,MEXICO,3.787431,5.034386e+11,0.248399
2,CHINA,19.998897,4.294258e+11,10.865955
3,CANADA,8.125310,4.116938e+11,0.100472
4,JAPAN,14.332213,1.513400e+11,1.508006
5,VIETNAM,14.351930,1.413612e+11,3.774345
6,"KOREA, SOUTH",13.307981,1.304587e+11,0.190750
7,TAIWAN,8.823285,1.152741e+11,0.938144
8,INDIA,33.004769,8.773522e+10,2.400663
9,UNITED KINGDOM,10.950574,6.820597e+10,0.956848


In [7]:
def make_empty_df():

    empty_df = pd.DataFrame(columns=['country_name',"2025 tariff increase", 'total imports',"2024 tariff",])

    empty_df.loc[0, 'country_name'] = ''
    
    empty_df.loc[0, 'total imports'] = 0.0

    empty_df.loc[0, '2025 tariff increase'] = 0.0

    empty_df.loc[0, '2024 tariff'] = 0.0

    return empty_df

In [8]:
def make_source(df):
    
    df["position"] = df.reset_index().index.values
        
    df["hover_label"] = (df["total imports"]/1000000000).map('{:,.1f}'.format)
        
    df["hover_label_2"] = (df["2025 tariff increase"]).map('{:,.1f}'.format)
    
    df["hover_label_3"] = (df["2024 tariff"]).map('{:,.1f}'.format)

    source = ColumnDataSource(df)
    
    return source

In [9]:
def make_bar_chart(df):

    height = int(1.15*533)
    width = int(1.15*750)

    source = make_source(df)
        
    p = figure(plot_height=height, plot_width = width, 
               title= "U.S. Tariff Increase Since Jan 20th by Country of Origin (Top 20 by Import Value)", x_range=df['country_name'],
           toolbar_location = 'below',
           tools = "reset")
            
    p.vbar(x = "country_name", top = "2025 tariff increase", width = 0.6, alpha = 0.65,
       hatch_pattern = " ",hatch_alpha = 0.10, color = "red",
       source = source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)

##########################################################################
    TIMETOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95; border: 0px 0px 0px 0px">
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">@country_name</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">2024 Imports: $@hover_label Billion</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Applied Tariff Increase: @hover_label_2%</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">2024 Applied Tariff: @hover_label_3%</span>
        </div>
    </div>
    """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    df["position"] = df.reset_index().index.values
    mid_idx = len(df['position']) // 2
    x_val = df['position'].iloc[mid_idx]
    y_val = df["2025 tariff increase"].max() * .85

    mytext = Label(x=x_val, y=y_val, text='U.S. Average Tariff:', 
               text_font_size="2em", text_font_style="bold",
               x_units='data', y_units='data', render_mode='canvas')

    # p.add_layout(mytext)

    mytext2 = Label(x=x_val, y=y_val * 0.75, text=str(round(avg_tariff,1)) + '%',
                text_font_size="6em", text_font_style="bold",
                x_units='data', y_units='data', render_mode='canvas')

    # p.add_layout(mytext2)

    #p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '14pt'
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks


    p.xaxis.major_label_text_font_size = '7pt'  # turn off x-axis tick labels
    p.xaxis.major_label_orientation = 0.75 

    p.yaxis.formatter = NumeralTickFormatter(format="(0. a)")
    p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    
    p.y_range.end = df["2025 tariff increase"].max() + 0.10*df["2025 tariff increase"].max()
    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)

    
    return p

In [10]:
crl = ["darkblue","slategray","slategray","crimson","crimson"]

background = "#ffffff"

In [11]:
fed_tax = pd.read_csv("federal-tax-duty.csv",skiprows=3, nrows=17)
# from Table 3.5. Taxes on Production and Imports

fed_duties = fed_tax.iloc[15,2:]

# Convert the Series to a DataFrame
fed_duties_df = fed_duties.to_frame()

# # Optionally, rename the column for clarity
fed_duties_df.columns = ['duty']

fed_duties_df.index.rename('time', inplace=True)

# # # Reset the index if needed
fed_duties_df.reset_index(inplace=True)

fed_duties_df["time"] = pd.to_datetime(fed_duties_df["time"], format="%Y")

# Extract only the year and overwrite the 'time' column
fed_duties_df["time"] = fed_duties_df["time"].dt.year

######################################################################################

NIPA_imports = pd.read_csv("NIPA-imports.csv",skiprows=3, nrows=20)
# from Table 4.1. Foreign Transactions in the National Income and Product Accounts

imports = NIPA_imports.iloc[19,2:]

imports_df = imports.to_frame()

# Optionally, rename the column for clarity
imports_df.columns = ['import_value']

# Rename the index
imports_df.index.rename('time', inplace=True)

# Reset the index if needed
imports_df.reset_index(inplace=True)

# Ensure the 'time' column is in datetime format
imports_df["time"] = pd.to_datetime(imports_df["time"], format="%Y")

# Extract only the year and overwrite the 'time' column
imports_df["time"] = imports_df["time"].dt.year

########################################################################################

merged_df = pd.merge(fed_duties_df, imports_df, on='time', how='inner')

merged_df["duty"] = merged_df["duty"].astype(float)
merged_df["import_value"] = merged_df["import_value"].astype(float)

merged_df["tariff"] = ( merged_df["duty"] / merged_df["import_value"] ) * 100

# Create a new row with the year 2024 and a tariff value
# comes from www.tradewartracker.com
new_row = {'time': "2024", 'duty': 0, 'import_value': 0, 'tariff': 2.4}

# Append the new row to the DataFrame
merged_df = merged_df.append(new_row, ignore_index=True)

# Create a new row with the year 2024 and a tariff value
# comes from www.tradewartracker.com
new_row = {'time': "2025", 'duty': 0, 'import_value': 0, 'tariff': avg_tariff}

# Append the new row to the DataFrame
merged_df = merged_df.append(new_row, ignore_index=True)

# Ensure the 'time' column is in datetime format
merged_df["time"] = pd.to_datetime(merged_df["time"], format="%Y")

# # Extract only the year and overwrite the 'time' column
# merged_df["time"] = merged_df["time"].dt.year

In [12]:
avg_tariff

13.975266365491787

In [13]:
def make_tariff_time(df):

    height = int(1.15*533)
    width = int(1.15*750)

    source = ColumnDataSource(df)
        
    p = figure(x_axis_type="datetime", plot_height=height, plot_width = width, title= "U.S. Tariff Rate 1929-2025",
           toolbar_location = 'below',
           tools = "reset")
            
    p.line(x = "time", y = "tariff",
                line_width=10, line_alpha=0.75, line_color = "slategray",
                hover_line_alpha=0.75, hover_line_width = 12,
                hover_line_color= "crimson", source = source)
    
    # p.circle(
    #     x=pd.to_datetime("2025"),  # Specify the x-coordinate (time)
    #     y=22.5,                     # Specify the y-coordinate (tariff value)
    #     size=15,
    #     alpha = 0.75,            # Transparency of the circle
    #     color="red",               # Color of the circle
    # )
    
##########################################################################
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> Average Tariff
             </span>
             </div>
             <div style = "text-align:left;">"""
    

    TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%Y}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """
        
    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))

    # p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '16pt'

    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

    p.x_range.start = pd.to_datetime("1928-01-01")
    p.xaxis.major_label_text_font_size = '12pt'  # turn off x-axis tick labels
    p.xaxis.major_label_orientation = 0.75 

    p.yaxis.formatter = NumeralTickFormatter(format="(0. a)")
    # p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    p.y_range.end = 30.0 

    p.yaxis.major_label_text_font_size = "12pt"  # Increase font size
    
    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)

    
    return p

In [14]:
daily_tariff = pd.read_csv("daily-tariff-latest-data.csv", usecols=["date", "import_weighted_avg_tariff"])

daily_tariff["date"] = pd.to_datetime(daily_tariff["date"])

# 1. Set date as index if not already
daily_tariff = daily_tariff.set_index('date')

# # 2. Create a regular date range (e.g., daily)
# full_range = pd.date_range(daily_tariff.index.min(), pd.Timestamp.now(), freq='D')

# # 3. Reindex and forward-fill missing values
# daily_tariff_filled = daily_tariff.reindex(full_range).ffill()

# # 4. Reset index for plotting
# daily_tariff_filled = daily_tariff_filled.reset_index().rename(columns={'index': 'date'})


In [15]:
pd.Timestamp.now().floor('D')

Timestamp('2025-12-11 00:00:00')

In [16]:
daily_tariff.loc[pd.Timestamp.now():"2025-08-30"]

,import_weighted_avg_tariff
date,


In [17]:
def make_tariff_daily(df):

    height = int(1.15*533)
    width = int(1.15*750)

    source = ColumnDataSource(df)

    current_tariff = df['import_weighted_avg_tariff'].iloc[-1]
        
    p = figure(x_axis_type="datetime", plot_height=height, plot_width = width, 
               title= "Today's Tariff Rate: " + str(round(avg_tariff,1)) + "%",
           toolbar_location = 'below',
           tools = "reset")

    source = ColumnDataSource(df.loc["2025-01":pd.Timestamp.now().floor('D')])

    p.line(x = "date", y = "import_weighted_avg_tariff",
                line_width=10, line_alpha=0.75, line_color = "crimson",
                hover_line_alpha=0.75, hover_line_width = 12,
                hover_line_color= "crimson", source = source)

    source = ColumnDataSource(df.loc[pd.Timestamp.now().floor('D'):"2025-12-31"])


    # source = ColumnDataSource(df.loc["2025-08-05":"2025-08-07"])
            
    p.line(x = "date", y = "import_weighted_avg_tariff",
                line_width=10, line_alpha=0.75, line_color = "darkblue",
                hover_line_alpha=0.75, hover_line_width = 12,
                hover_line_color= "darkblue", source = source)
    
    # source = ColumnDataSource(df.loc[["2025-08-01"]])
            
    # # p.line(x = "date", y = "import_weighted_avg_tariff",
    # #             line_width=10, line_alpha=0.75, line_color = "darkblue",
    # #             hover_line_alpha=0.75, hover_line_width = 12,
    # #             hover_line_color= "darkblue", source = source)
    
    # p.circle(x = "date", y = "import_weighted_avg_tariff",
    #             line_width=10, line_alpha=0.75, line_color = "darkblue",
    #             hover_line_alpha=0.75, hover_line_width = 12,
    #             hover_line_color= "darkblue", source = source)
    
    
##########################################################################
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> Average Tariff
             </span>
             </div>
             <div style = "text-align:left;">"""
    

    TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%Y-%m-%d}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """
        
    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))

    # p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '20pt'

    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

    p.x_range.start = pd.to_datetime("2025-01-01")
    p.xaxis.major_label_text_font_size = '12pt'  # turn off x-axis tick labels
    p.xaxis.major_label_orientation = 0.75 

    p.yaxis.formatter = NumeralTickFormatter(format="(0. a)")
    # p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    p.y_range.end = 35.0 

    p.yaxis.major_label_text_font_size = "12pt"  # Increase font size

    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)

    y_val = df['import_weighted_avg_tariff'].iloc[-1]

    # x_val = pd.to_datetime("2025-01-20")

    # mytext = Label(x=x_val, y=y_val*1.2, text='Current U.S. Average Tariff:', 
    #            text_font_size="2em", text_font_style="bold",
    #            x_units='data', y_units='data', render_mode='canvas')

    # p.add_layout(mytext)

    # current_tariff = df['import_weighted_avg_tariff'].iloc[-1]

    # mytext2 = Label(x=x_val, y=y_val * 0.75, text=str(round(current_tariff,1)) + '%',
    #             text_font_size="6em", text_font_style="bold",
    #             x_units='data', y_units='data', render_mode='canvas')

    # p.add_layout(mytext2)

    
    return p

In [18]:
# foo = recipricol_summary[0:20].copy(deep=True)

# foobar = make_empty_df()

# foobar = foobar.append(foo)

foo = recipricol_summary.loc["2025-11-14"].copy(deep=True)

foo.sort_values(by='total imports', ascending=False, inplace=True)

foo[0:20].head(20)

foobar = foo[0:20].copy(deep=True)

foobar.rename({"effective tariff": "2025 tariff increase"}, axis=1, inplace=True)

foobar.reset_index(inplace=True, drop=True)

bar = make_empty_df()

foobar = bar.append(foobar)

p1 = make_bar_chart(foobar)


####################################################


# foo = recipricol_summary.loc["2025-11-01"].copy(deep=True)

# foo.sort_values(by='total imports', ascending=False, inplace=True)

# foo[0:20].head(20)

# foobar = foo[0:20].copy(deep=True)

# foobar.rename({"effective tariff": "2025 tariff increase"}, axis=1, inplace=True)

# foobar.reset_index(inplace=True, drop=True)

# bar = make_empty_df()

# foobar = bar.append(foobar)

# p4 = make_bar_chart(foobar)

####################################################


p2 = make_tariff_time(merged_df)

p3 = make_tariff_daily(daily_tariff)

# tab2 = Panel(child= p2, title="Tariffs Over Time")

output_file('.\\docs\\' + "reciprocal-new-steel.html")

div1 = Div(text = """Each bar represents the increase in the trade-weighted applied tariff for each country since January 20th including July 31st EO
        and known deals (EU, Japan, etc.). Fentanyl tariffs for Canada and Mexico are calculated under the assumption that all auto 
           imports are USMCA-compliant, and that 65% of Canadian imports and 82% of Mexican imports are USMCA-compliant. The average tariff 
           reflects the trade-weighted average of all tariffs, including the existing 2.3% applied tariff as of February 2025.
""", max_width= int(1.15*750), background = background )

div4 = Div(text = """Each bar represents the increase in the trade-weighted applied tariff for each country since January 20th 
           **not** including July 31st EO and known deals (EU, Japan, etc.). Fentanyl tariffs for Canada and Mexico are calculated under the assumption that all auto 
           imports are USMCA-compliant, and that 65% of Canadian imports and 82% of Mexican imports are USMCA-compliant. The average tariff 
           reflects the trade-weighted average of all tariffs, including the existing 2.3% applied tariff as of February 2025.
""", max_width= int(1.15*750), background = background )


div2 = Div(text = """This chart shows the average U.S. tariff rate from 1929 to the present.
The average tariff is calculated as collected duties (from NIPA Table 3.5) divided by the value of goods imports (from NIPA Table 4.1).
The 2025 estimated tariff rate for 2025 includes July 31st reciprocal tariffs, steel and aluminum tariffs, auto tariffs,
and the fentanyl-related tariffs. These estimates account for exemptions where applicable.   
""", max_width= int(1.15*750), background = background )

div3 = Div(text = """This plot displays the average daily tariff rate since January 2025 including July 31st EO and known deals (EU, Japan, etc.). 
           The blue line projects out the tariff rate given the (10/30) deal with China and updated Annex II (Nov 14). Fentanyl tariffs for Canada and Mexico are calculated under the assumption that all auto 
           imports are USMCA-compliant, and that 65% of Canadian imports and 82% of Mexican imports are USMCA-compliant. The average tariff 
           reflects the trade-weighted average of all tariffs, including the existing 2.3% applied tariff as of February 2025.  
""", max_width= int(1.15*750), background = background )

tab1_layout = column(p1, div1, sizing_mode="scale_both")
tab2_layout = column(p2, div2, sizing_mode="scale_both")
tab3_layout = column(p3, div3, sizing_mode="scale_both")
# tab4_layout = column(p4, div4, sizing_mode="scale_both")


# Create tabs
tab1 = Panel(child=tab1_layout, title="Tariffs by Country")
tab2 = Panel(child=tab2_layout, title="Tariffs Over Time")
tab3 = Panel(child=tab3_layout, title="Daily Tariff Rate")
# tab4 = Panel(child=tab4_layout, title="Tariffs by Country (Pre-August 7th)")

# Combine tabs into a Tabs layout
tabs = Tabs(tabs=[tab3, tab1, tab2], tabs_location="above")

show(tabs)

In [19]:
foobar.head(20)

,country_name,2025 tariff increase,total imports,2024 tariff,position,hover_label,hover_label_2,hover_label_3
0,,0.0,0.0,0.0,0,0.0,0.0,0.0
0,EUROPEAN UNION,11.129221,597473236377.0,1.229078,1,597.5,11.1,1.2
1,MEXICO,3.787431,503438605302.0,0.248399,2,503.4,3.8,0.2
2,CHINA,19.998897,429425828419.0,10.865955,3,429.4,20.0,10.9
3,CANADA,8.12531,411693824741.0,0.100472,4,411.7,8.1,0.1
4,JAPAN,14.332213,151340030810.0,1.508006,5,151.3,14.3,1.5
5,VIETNAM,14.35193,141361159091.0,3.774345,6,141.4,14.4,3.8
6,"KOREA, SOUTH",13.307981,130458735626.0,0.19075,7,130.5,13.3,0.2
7,TAIWAN,8.823285,115274122243.0,0.938144,8,115.3,8.8,0.9
8,INDIA,33.004769,87735219365.0,2.400663,9,87.7,33.0,2.4
